In [1]:
#-*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import math
from collections import Counter
import ast
from konlpy.tag import Hannanum
# from konlpy.tag import Kkma
# from konlpy.tag import Komoran
# from konlpy.tag import Mecab
# from konlpy.tag import Twitter
from konlpy.utils import concordance, pprint
import datetime
import os
import re
import random
import time
import sys
reload(sys)
import psutil
sys.setdefaultencoding('utf-8')
from operator import itemgetter

In [2]:
import resource

In [3]:
def count_tags(tags):
    dic_tag = {}
    for el in tags:
        tag = el[1]
        if not tag in dic_tag:
            dic_tag[tag] = 1
        else:
            dic_tag[tag] += 1
    return dic_tag

#수식언, 체언, 용언만 딕셔너리로 리턴
#Pos == PartOfSpeech
def filter_pos(pos_of_article, tagger):
    if tagger == 'han':
        # 수식언, 체언, 용언
        filtered_pos = {'M':{}, 'N':{}, 'P':{}}
        permitted_pos = ['M','N','P']
#     elif tagger in ['komo', 'kkma']:
#         # 고유명사, 보통명사, 대명사, 수사, 형용사, 동사, 어근
#         dic_tag = {'NNG':{}, 'NNP':{},'NP':{},'NR':{},'VA':{},'VV':{},'XR':{}}
#         arr_tag = ['NNG','NNP','NP','NR','VA','VV','XR']
    for el in pos_of_article:
        word = el[0]
        pos = el[1]
        if pos in permitted_pos:
            if word in filtered_pos[pos]:
                filtered_pos[pos][word] += 1
            else:
                filtered_pos[pos][word] = 1
    return filtered_pos

def processing_words(dic):
    processed_hannanum = {}
    for key in dic.keys():
        x = sorted(dic[key].items(), key=itemgetter(1), reverse = True)
        processed_hannanum[key] = x
    return processed_hannanum

def print_result(result):
    if type(result) == list:
        for item in result:
            pprint(item)
    else:
        for key, value in result.items():
            pprint(key)
            pprint(value)

def get_total_words(dic):
    total = 0
    for tag in dic.keys():
        dic_specific_tag = dic[tag]
        total += sum(dic_specific_tag.values())
    return total

def get_unique_words(dic):
    total = 0
    for tag in dic.keys():
        dic_specific_tag = dic[tag]
        total += len(dic_specific_tag)
    return total

In [4]:
def calculate_score(dic1, dic2, total_dic, algorithm):
    same_words = 0
    total_words = 0
    similarity_score = 0
    if algorithm == 1:
        #same words between dic1 and dic2 / total_words of dic1 and dic2
        for tag in dic1.keys(): #['M', 'N', 'P']
            tagged_words_from_dic1 = dic1[tag]
            tagged_words_from_dic2 = dic2[tag]
            for word in tagged_words_from_dic1.keys():
                if word in tagged_words_from_dic2:
                    word_count = tagged_words_from_dic1[word] + tagged_words_from_dic2[word]
                    same_words += word_count
        total_words = get_total_words(dic1) + get_total_words(dic2)
        similarity_score = float(same_words) / float(total_words)

        print('similarity score:', similarity_score)
        return round(similarity_score, 3)

    elif algorithm == 2:
        #unique same words between dic1 and dic2 / unique total words of total dic(dic1+dic2)
        for tag in dic1.keys():
            dic1_specific_tag = dic1[tag]
            dic2_specific_tag = dic2[tag]
            for word in dic1_specific_tag.keys():
                if word in dic2_specific_tag:
                    same_words += 1
        total_words = get_unique_words(total_dic)
        similarity_score = float(same_words) / float(total_words)

        print('similarity score:', similarity_score)
        return round(similarity_score, 3)

In [35]:
class calculate_similarity_of():

    def __init__(self, df):
        df = df.dropna(subset=['Content']).reset_index(drop=True)
        df = df.sort_values('Date').reset_index(drop=True)
        df['pos_tagged_words'] = None
        self.df = df

    def _get_pos_tagged_words_(self, article, index):
        if index == None:
            pos_tagged_words = None
            content = article
        else:
            pos_tagged_words = self.df.pos_tagged_words[index]
            content = article.Content

        if pos_tagged_words == None:
            #print("Update for acticle", index, "started")
            pos_article = Hannanum().pos(content.replace(' \n', '').replace('\n ',''))
            #print('Pos is tagged for article')
            pos_tagged_words = filter_pos(pos_article, 'han')
            #print('Pos is filtered')
            if index != None:
                self.df.loc[index, 'pos_tagged_words'] = repr(pos_tagged_words)
                #print('pos_tagged_words_saved')
                print("Get article", index, "'s pos_tagged_words")
        else:
            pos_tagged_words = ast.literal_eval(pos_tagged_words)
            print("Get article", index, "'s pos_tagged_words")
        return pos_tagged_words
    
    
    def _find_articles_to_compare_(self, i):
        needed_article_number = 10
        index_list = []
        finding_articles = True
        yesterday = self.df.Date[i] - datetime.timedelta(1)
        firstday = self.df.Date[0]
        while finding_articles:
            yesterday_articles = self.df[self.df.Date == yesterday]
            if len(yesterday_articles) == 0:
                yesterday -= datetime.timedelta(1)
                #print('Keep going back to', yesterday)
                if yesterday < firstday:
                    yesterday_articles = self.df.loc[i-needed_article_number:i-1]
                    index_list += list(yesterday_articles.index)
                    finding_articles = False
                continue
            elif len(yesterday_articles) >= needed_article_number:
                index_list += random.sample(yesterday_articles.index, needed_article_number)
                finding_articles = False
                #print('Found all articles to compare')
            else:
                index_list += list(yesterday_articles.index)
                needed_article_number -= len(yesterday_articles)
                yesterday -= datetime.timedelta(1)
                #print('Found', len(yesterday_articles), 'articles and need more')
        return self.df.iloc[index_list]
    
    
    def on(self, from_number, to_number):
        process = psutil.Process(os.getpid())
        article_df = self.df
        start = time.time()
        for i in range(from_number, to_number):
            print('from', from_number, 'to', to_number)
            start_i = time.time()
            current_article = article_df.iloc[i]
            compared_articles = self._find_articles_to_compare_(i)
            j = 1
            for ci, compared_article in compared_articles.iterrows():
                start_j = time.time()
                total_article_content = current_article.Content + compared_article.Content

                current_article_words = self._get_pos_tagged_words_(current_article, i)
                compared_article_words = self._get_pos_tagged_words_(compared_article, ci)
                total_article_words = self._get_pos_tagged_words_(total_article_content, None)

                similarity_score1 = calculate_score(current_article_words, compared_article_words, total_article_words, 1)
                similarity_score2 = calculate_score(current_article_words, compared_article_words, total_article_words, 2)
 
                article_df.loc[i, 'si_1_{j}'.format(j=j)] = similarity_score1
                article_df.loc[i, 'si_2_{j}'.format(j=j)] = similarity_score2
                j += 1
                end_j = time.time()
                print('comparing one article takes', end_j - start_j, 'seconds', 'now:', str(datetime.datetime.today()))
                print('total_cpu:', psutil.cpu_percent(percpu=True),\
                      'process_cpu:', process.cpu_percent())

                print('p_using_memory:',process.memory_full_info().rss/1024/1024, \
                      'p_using_v_memory:', process.memory_full_info().vms/1024/1024)

                print('total_memory:', psutil.virtual_memory().total/1024/1024,\
                      'available_memory:', psutil.virtual_memory().available/1024/1024,
                      'using_percent:', psutil.virtual_memory().percent)
            average1 = article_df.loc[i, ['si_1_{j}'.format(j=j) for j in range(1,11)]].mean()
            average2 = article_df.loc[i, ['si_2_{j}'.format(j=j) for j in range(1,11)]].mean()
            article_df.loc[i, 'average1'] = average1
            article_df.loc[i, 'average2'] = average2
            end_i = time.time()
            print('comparing 10 articles takes', end_i-start_i, 'seconds')
        end = time.time()
        print(end-start)

In [36]:
folder = '/home/ubuntu/ubuntu/'
files = os.listdir(folder)

files = filter(lambda x: 'xlsx' in x , files)

number = re.compile('\d{3,5}(?=.xlsx)')

number_and_files = []
for f in files:
    try:
        count = int(number.findall(f)[0])
        number_and_files.append([count, f])
    except:
        continue

file_addr_sorted = sorted(number_and_files, key = itemgetter(0))

In [82]:
files = file_addr_sorted[8:20]

In [84]:
len(file_addr_sorted)

54

In [ ]:
for fi in files:
    file_address = folder + fi[1]
    address_for_save = folder + 'complete/' + fi[1][:-5] + '_completed.xlsx'
    print('read', file_address, 'will be saved in', address_for_save)
    article_df = pd.read_excel(file_address)
    call = calculate_similarity_of(article_df)
    call.on(10, len(call.df))
    call.df.to_excel(address_for_save)